# Modify the RGI6 regions files for RGI7 

List of changes:
- Region 12 (Caucasus and Middle East): there is a cluster of glaciers south of the current extent of the region and subregion polygons. There are no regions below them, so there shouldn't be much of an issue in simply updating the geometry a little bit: we shift the southern boundary by 2° (from 32°N to 30°N).
- the data type of the RGI_CODE attribute in the region file of version 6 is int. For consistency with the RGI files, it should be str (format with leading zero, for example `02`). We change this as well.

In [ ]:
# go down from rgi7_scripts/workflow/preprocessing
data_dir = '../../../rgi7_data/'

In [ ]:
import os
import numpy as np
import shapely.geometry as shpg
import geopandas as gpd
from utils import mkdir

## Regions 

In [ ]:
out_dir = os.path.abspath(os.path.join(data_dir, '00_rgi70_regions'))
mkdir(out_dir)

In [ ]:
# Read the RGI region files
rgi_dir = os.path.join(data_dir, 'l0_RGIv6')
rgi_reg = gpd.read_file('zip://' + os.path.join(data_dir, 'l0_RGIv6', '00_rgi60_regions.zip', '00_rgi60_O1Regions.shp'))

In [ ]:
# Select the RGI 12 polygon
poly = rgi_reg.loc[rgi_reg.RGI_CODE == 12].iloc[0].geometry
poly.bounds

Let's go down to 30° South instead:

In [ ]:
x, y = poly.exterior.xy
ny = np.where(np.isclose(y, 31), 30, y)
new_poly = shpg.Polygon(np.array((x, ny)).T)
rgi_reg.loc[rgi_reg.RGI_CODE == 12, 'geometry'] = new_poly

In [ ]:
# Change type and format
rgi_reg['RGI_CODE'] = ['{:02d}'.format(int(s)) for s in rgi_reg.RGI_CODE]

In [ ]:
rgi_reg

In [ ]:
rgi_reg.to_file(os.path.join(out_dir, '00_rgi70_O1Regions.shp'))

In [ ]:
# Check
rgi_reg = gpd.read_file(os.path.join(out_dir, '00_rgi70_O1Regions.shp'))
assert rgi_reg.RGI_CODE.dtype == 'O'

## Subregions 

In [ ]:
rgi_reg = gpd.read_file('zip://' + os.path.join(data_dir, 'l0_RGIv6', '00_rgi60_regions.zip', '00_rgi60_O2Regions.shp'))

In [ ]:
poly = rgi_reg.loc[rgi_reg.RGI_CODE == '12-02'].iloc[0].geometry
poly.bounds

In [ ]:
x, y = poly.exterior.xy
ny = np.where(np.isclose(y, 32), 30, y)
new_poly = shpg.Polygon(np.array((x, ny)).T)
rgi_reg.loc[rgi_reg.RGI_CODE == '12-02', 'geometry'] = new_poly

In [ ]:
rgi_reg.to_file(os.path.join(out_dir, '00_rgi70_O2Regions.shp'))

In [ ]:
rgi_reg